In [72]:
from pathlib import Path
import numpy as np, plotly.express as px, pandas as pd
h = 500

diff = 20
labels = {0:'angle', 1:'info->rotate_angle_step', 2:'xamp', 3:'info->xamp_step'}
ar = np.array([[float(j) for j in i.split()] for i in Path('graph').read_text().replace('\n\n', '\n').replace('\n\n', '\n').splitlines() if i])



def plot(arr):
    arr = np.array(arr)
    fig = px.line(arr, height=300)
    fig.update_layout(
        xaxis=dict(range=[0, len(arr)-1]),
        yaxis=dict(range=[arr.min(), arr.max()]),
        margin=dict(l=0, r=0, t=10, b=0)
    )
    fig.show()

def quan(arr: list, per=0.05, lr=0.02, max_iter=40):

    n = len(arr)
    target = n * (1 - per)

    low, high = min(arr), max(arr)
    thres = (high - low) * per + low  # initial guess ~per percentile
    print(np.quantile(arr, per))
    for _ in range(max_iter):
        # how many elements are above current threshold?
        count = sum(1 for i in arr if i > thres)

        # error = how far off we are from target
        error = count - target
        if abs(error) < 20:  # good enough (within 1 element)
            break

        print(round(thres, 3), round(count/len(arr), 4), round(error, 2), round(lr, 2))

        # gradient = sign of error (since F(t) is monotonic decreasing)
        grad = 1 if error > 0 else -1

        # update threshold
        thres += error * lr# * (high - low)

        # shrink learning rate for stability
        lr *= 0.95

    return thres


def count_blades(arr, derivative_jump=10, ema=0.95, quantile=.95):

    print('original:')
    plot(arr)

    print(f'clipped {quantile}:')
    top = np.quantile(arr, quantile)
    bottom = np.quantile(arr, 1-quantile)
    arr = [top if i > top else i if i > bottom else bottom for i in arr]
    plot(arr)

    print(f'averaged {ema} ema')
    for i in range(1, len(arr)):
        arr[i] = arr[i-1] * ema + (1-ema)*arr[i]
    plot(arr)

    print(f'derivative jump: {derivative_jump}')
    for i in range(len(arr)-derivative_jump):
        arr[i] = arr[i+derivative_jump] - arr[i]
    arr = arr[:-derivative_jump]
    plot(arr)

    print('clipped below zero')
    arr = [i if i > 0 else 0 for i in arr]
    plot(arr)

    quantile -= .05
    top = np.quantile(arr, quantile)
    print(f'quantile {quantile} {top}')
    arr = [i if i < top else top for i in arr]
    plot(arr)

    print('binary')
    middle = (max(arr) + min(arr)) * .5
    arr = [i > middle for i in arr]
    plot(arr)

    count = 0
    for i in range(1, len(arr)-1):
        if not arr[i-1] and arr[i]:
            count += 1
    print(f"counted: {count}")

    return count

count_blades(ar[:,0])

original:


clipped 0.95:


averaged 0.95 ema


derivative jump: 10


clipped below zero


quantile 0.8999999999999999 3.340750448056701


binary


counted: 35


35

In [109]:
1/(4095.0 * 0.707106781)

0.00034535129737391703

In [67]:
1/(5000*25/1e6)
20/(4592*25/60e6)
15*8

120